In [72]:
import scipy.io as sio
from scipy.misc import imread
from preprocess.normalize import preprocess_signature
import tf_signet
from tf_cnn_model import TF_CNNModel
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import random
from numpy.random import choice
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import sklearn.pipeline as pipeline
import sklearn.preprocessing as preprocessing

In [73]:
data_folder = 'C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\downloaded_pp_features\\cedar_signet'

In [74]:
data_f = pd.read_csv(os.path.join(data_folder,'data_features.csv'))
visual_f = pd.read_csv(os.path.join(data_folder,'visual_features.csv'))

In [75]:
dev_df_gen.shape

(24, 3)

# MODEL SELECTION & TRAINING

In [76]:
fakes_preds = []
gens_preds = []

for fold in np.arange(0,5):
    user_numbers = data_f['user_id'].unique()
    np.random.shuffle(user_numbers)
    dev_user_ids = user_numbers
    validation_user_ids = user_numbers

    dev_df = data_f.loc[data_f['user_id'].isin(dev_user_ids)]
    dev_vf = visual_f.loc[dev_df.index]
    val_df = data_f.loc[data_f['user_id'].isin(validation_user_ids)]
    val_vf = visual_f.loc[val_df.index]

    dev_df_gen = dev_df.loc[dev_df['fakeness']==0]
    dev_df_fake = dev_df.loc[dev_df['fakeness']==1]
    dev_df_gen_12 = dev_df_gen.loc[dev_df_gen['sig_id'].isin(np.arange(1,13))]
    dev_df_valid_12 = dev_df_gen.loc[dev_df_gen['sig_id'].isin(np.arange(13,25))]

    val_df_gen = val_df.loc[val_df['fakeness']==0]
    val_df_fake = val_df.loc[val_df['fakeness']==1]
    val_df_gen_12 = val_df_gen.loc[val_df_gen['sig_id'].isin(np.arange(1,13))]
    val_df_valid_gen_12 = val_df_gen.loc[val_df_gen['sig_id'].isin(np.arange(13,25))]


    for user_id in tqdm.tqdm(validation_user_ids, ascii=True):
        clf = SVC(C=1,gamma=2**-11,class_weight='balanced', probability=False, kernel='rbf', random_state=1923)
        # clf = pipeline.Pipeline([('scaler', preprocessing.StandardScaler(with_mean=False)), ('classifier', orig_clf)])
        y_train = (pd.concat([val_df_gen_12.loc[val_df_gen_12['user_id']==user_id],dev_df_gen]))['user_id']==user_id
        X_train = visual_f.loc[y_train.index]  
        clf.fit(X_train, y_train)
        y_valid_fakes = val_df_fake.loc[(val_df_fake['user_id']==user_id)]
        X_valid_f = visual_f.loc[y_valid_fakes.index]
        fakes_preds.append(clf.decision_function(X_valid_f))
        y_valid_gens = val_df_valid_gen_12.loc[val_df_valid_gen_12['user_id']==user_id]
        X_valid_g = visual_f.loc[y_valid_gens.index]
        gens_preds.append(clf.decision_function(X_valid_g))


100%|##################################################################################| 55/55 [00:14<00:00,  3.21it/s]


# THRESHOLD SELECTION

In [77]:
DECISION_THRESHOLD = 0.8
tn = 0
fp = 0
for val in fakes_preds:
    for pred in val:
        if pred <= DECISION_THRESHOLD: tn += 1
        else : fp += 1

print("fakes error rate:",(1-(float(tn)/(tn+fp)))*100)
        
tp = 0
fn = 0
for val in gens_preds:
    for pred in val:
        if pred >= DECISION_THRESHOLD: tp += 1
        else : fn += 1

print("gens error rate:",(1-(float(tp)/(tp+fn)))*100)

fakes error rate: 2.348484848484844
gens error rate: 0.0
